In [2]:
import awkward as ak
import matplotlib.pyplot as plt
import os, sys
import subprocess
import json
import uproot
from coffea.nanoevents import NanoEventsFactory, ScoutingNanoAODSchema
from coffea.lookup_tools.lookup_base import lookup_base
import numpy as np
from coffea import processor, util
from hist import Hist
import hist
from coffea.analysis_tools import Weights, PackedSelection
from collections import defaultdict
import mplhep
plt.style.use(mplhep.style.CMS)

In [ ]:
class Processor(processor.ProcessorABC):
    def __init__(self, wp_btag=0):
        self._wp_btag = wp_btag
        
    @property
    def accumulator(self):
        return {
            "sumw": defaultdict(float),
            "cutflow": defaultdict(int),
            "templates": (
                            Hist.new.StrCategory(
                                [], name="dataset", label="Dataset", growth=True
                            ).StrCategory(
                                [], name="region", label="Region", growth=True
                            ).StrCategory(
                                [], name="cutflow", label="Cutflow", growth=True
                            ).StrCategory(
                                [], name="cat", label="Category", growth=True
                            ).Reg(
                                50, 100, 700, name="pt", label=r"$p_T$"
                            ).Reg(
                                50, 0, 150, name="mass", label=r"mass"
                            ).Reg(
                                50, 0, 150, name="mreg", label=r"mreg"
                            ).Reg(
                                50, 0, 1, name="ddb", label=r"ddb"
                            ).Double()
                        ),
        }
           
        
    def process(self, events):
        
        output = self.accumulator
        dataset = events.metadata['dataset']
        
        isRealData = not hasattr(events, "genWeight")
        isQCDMC = 'QCD' in dataset
        
        selection = PackedSelection()
        
        if not isRealData:
            output['sumw'][dataset] += ak.sum(events.genWeight)
            
        if len(events) == 0:
            return output
        
        fatjets = events.ScoutingFatJet
        fatjets["pn_Hbb"] = ak.where((fatjets.particleNet_prob_Hbb + fatjets.particleNet_prob_QCD) == 0, 0, (fatjets.particleNet_prob_Hbb / (fatjets.particleNet_prob_Hbb + fatjets.particleNet_prob_QCD)))
        jets = events.ScoutingJet
        jets["pn_b"] = ak.where((jets.particleNet_prob_b + jets.particleNet_prob_g) == 0, 0, (jets.particleNet_prob_b / (jets.particleNet_prob_b + jets.particleNet_prob_g)))
        
        # trigger
        selection.add("trigger", events.HLT["Mu50"])
        
        # require MET
        met = events.ScoutingMET
        selection.add('met', met.pt > 50)
        
        # require at least one good muon
        goodmuon = (
            (events.ScoutingMuon.pt > 55)
            & (abs(events.ScoutingMuon.eta) < 2.4)
            & (abs(events.ScoutingMuon.trk_dxy) < 0.2)
            #& (abs(events.ScoutingMuon.trk_dz) < 0.5)
            #& (events.ScoutingMuon["type"] == 2)
            & (events.ScoutingMuon.normchi2 < 10)
            & (events.ScoutingMuon.nValidRecoMuonHits > 0)
            & (events.ScoutingMuon.nRecoMuonMatchedStations > 1)
            & (events.ScoutingMuon.nValidPixelHits > 0)
            & (events.ScoutingMuon.nTrackerLayersWithMeasurement > 5)            
        )
        
        nmuons = ak.sum(goodmuon, axis=1)
        selection.add('onemuon', (nmuons > 0))
        
        # require good leptonic W 
        leadingmuon = ak.firsts(events.ScoutingMuon[goodmuon])
        leptonicW = met + leadingmuon
        selection.add('leptonicW', leptonicW.pt > 100)
        
        # require at least one b-jet in the same hemisphere of the leading muon
        dphi = abs(jets.delta_phi(leadingmuon))
        jetsamehemisp = jets[dphi < 2]
        bjets = (jetsamehemisp.pn_b > self._wp_btag)
        nbjets = ak.sum(bjets, axis=1)
        selection.add('onebjet', (nbjets > 0))

        # require fatjet away from the leading muon
        dphi = abs(fatjets.delta_phi(leadingmuon))
        is_away = (dphi > 2)
        nfatjets = ak.sum(is_away, axis=1)
        selection.add('onefatjet', (nfatjets > 0))
        
        proxy = ak.firsts(fatjets[is_away])
            
        regions = {
            'all': ['trigger','met','onemuon','leptonicW','onebjet','onefatjet'],
            'noselection': [],
        }
        
        def normalize(val, cut):
            if cut is None:
                ar = ak.to_numpy(ak.fill_none(val, np.nan))
                return ar
            else:
                ar = ak.to_numpy(ak.fill_none(val[cut], np.nan))
                return ar
            
        def fill(region, jet, _cut=None):
#             selections = regions[region]
#             cut = selection.all(*selections)
            cut = selection.all(_cut)

            left_after_cut = normalize(jet.pt, cut)
            output['cutflow'][_cut] += len(left_after_cut)
            
            top = jet["top_matched"].to_numpy()
            w = jet["w_matched"].to_numpy()
            non = jet["non_matched"].to_numpy()
            cat = []

            for i in range(len(top)):
                if top[i]:
                    cat.append("top_matched")
                elif w[i]:
                    cat.append("w_macthed")
                elif non[i]:
                    cat.append("non_matched")
            
            output['templates'].fill(
                dataset=dataset,
                region=region,
                cutflow=np.repeat(_cut, len(left_after_cut)),
                cat=np.array('cat'),
                pt=normalize(jet.pt, cut),
                mass=normalize(jet.mass, cut),
                mreg=normalize(jet.particleNet_mass, cut),
                ddb=normalize(jet.pn_Hbb, cut),
            )
            
        top = events.GenPart[
            (abs(events.GenPart.pdgId) == 6)
            & (events.GenPart.hasFlags(['fromHardProcess', 'isLastCopy']))
        ]
        
        w_top = top.distinctChildren[
            (abs(top.distinctChildren.pdgId) == 24)
            & (top.distinctChildren.hasFlags(['isLastCopy']))
        ]
        w_top = ak.flatten(w_top, axis=2)
        w_top_is_had = ak.any(
            (abs(w_top.distinctChildren.pdgId) < 7)
            & (w_top.distinctChildren.hasFlags(['isLastCopy']))
        , axis=2)
        
        had_top = w_top[w_top_is_had].distinctParent
        
        near_top, near_top_dr = proxy.nearest(had_top, axis=None, threshold=0.8, return_metric=True)
        near_top_dr = ak.to_numpy(ak.fill_none(near_top_dr, 99))
        
        b_near_top = near_top.distinctChildren[abs(near_top.distinctChildren.pdgId) == 5]
        b_near_top_matched, b_near_top_dr = proxy.nearest(b_near_top, axis=None, threshold=0.8, return_metric=True)
        b_near_top_dr = ak.to_numpy(ak.fill_none(b_near_top_dr, 99))

        W_near_top = near_top.distinctChildren[(
            (abs(near_top.distinctChildren.pdgId) == 24)
            & (ak.any(abs(near_top.distinctChildren.distinctChildren.pdgId) < 7))
        )]
        
        q_W_near_top = ak.flatten(W_near_top.distinctChildren, axis=2)
        q_W_near_top_dr = proxy.delta_r(q_W_near_top)
        
        index_ascend = ak.argsort(q_W_near_top_dr, axis=1)
        index_descend = ak.argsort(q_W_near_top_dr, ascending=False, axis=1)
        q_W_near_top_ascend = q_W_near_top[index_ascend]
        q_W_near_top_descend = q_W_near_top[index_descend]
        q_W_near_top_dr_ascend = q_W_near_top_dr[index_ascend]
        q_W_near_top_dr_descend = q_W_near_top_dr[index_descend]

        proxy["dr_T"] = near_top_dr
        proxy["dr_T_b"] = b_near_top_dr
        proxy["dr_T_Wq_max"] = ak.fill_none(ak.firsts(q_W_near_top_dr_descend), 99)
        proxy["dr_T_Wq_min"] = ak.fill_none(ak.firsts(q_W_near_top_dr_ascend), 99)
        proxy["dr_T_Wq_max_pdgId"] = ak.fill_none(ak.firsts(q_W_near_top_descend).pdgId, 99)
        proxy["dr_T_Wq_min_pdgId"] = ak.fill_none(ak.firsts(q_W_near_top_ascend).pdgId, 99)
        
        top_matched = (proxy["dr_T_b"] < 0.8) & (proxy["dr_T_Wq_max"] < 0.8)
        w_matched = (proxy["dr_T_b"] >= 0.8) & (proxy["dr_T_Wq_max"] < 0.8)
        non_matched = (~top_matched) & (~w_matched)
        
        cat = np.repeat("ERROR", len(proxy.pt))
        cat = ["top_matched" if t else c for c, t in zip(cat, top_matched.to_numpy())]
        cat = ["w_matched" if w else c for c, w in zip(cat, w_matched.to_numpy())]
        cat = ["non_matched" if n else c for c, n in zip(cat, non_matched.to_numpy())]
        proxy["cat"] = cat
            
        # at least one fatjet
        selection.add('min', ak.num(fatjets) > 0)
        fill('all', proxy, 'min')
            
        for region, cuts in regions.items():
            #fill(region, hadronictop)
            for cut in cuts:
                fill(region, proxy, cut)
            
        return output
    
    def postprocess(self, accumulator):
        return accumulator

In [ ]:
fileset = {
    "TTtoLNu2Q" : [
        "root://eosuser.cern.ch//eos/user/a/adlintul/TTtoLNu2Q_TuneCP5_13p6TeV_powheg-pythia8/Run3Summer22/230131_071621/0000/nanoaod_1.root"
    ],
}

p = Processor()

futures_run = processor.Runner(
    executor = processor.FuturesExecutor(compression=None, workers=2),
    schema = ScoutingNanoAODSchema,
    maxchunks = 10,
)

out = futures_run(
    fileset,
    treename="Events",
    processor_instance=p
)

In [ ]:
out

In [ ]:
import json

fileset = {}

with open("files.json") as fin:
    fileset = json.load(fin)

In [ ]:
from distributed import Client
from lpcjobqueue import LPCCondorCluster

from dask.distributed import performance_report
from dask_jobqueue import HTCondorCluster, SLURMCluster

env_extra = [
    f"export PYTHONPATH=$PYTHONPATH:{os.getcwd()}",
]

cluster = LPCCondorCluster(
    ship_env=True,
)

cluster.adapt(minimum=1, maximum=100)
client = Client(cluster)

print("Waiting for at least one worker...")
client.wait_for_workers(1)

uproot.open.defaults["xrootd_handler"] = uproot.source.xrootd.MultithreadedXRootDSource

output = processor.run_uproot_job(
            fileset,
            "Events",
            processor_instance=Processor(),
            executor=processor.dask_executor,
            executor_args={
                "schema": ScoutingNanoAODSchema,
                "savemetrics": True,
                "retries": 3,
                "client": client,
            },
            chunksize=10000,
            #maxchunks=args.max,
        )

In [ ]:
output

In [ ]:
output[0]['templates'].project('region','pt')[hist.loc('all'), :]

In [ ]:
out = output[0]

fig, axs = plt.subplots(2, 2)
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.4)

row = 0
column = 0
for i, var in enumerate(['pt', 'mass','mreg','ddb']):
    h = out['templates'].project('region',var)[hist.loc('all'), :]
    
    if i == 2:
        row = 0
        column = 1

    mplhep.histplot(h, label="Leading fatjet", ax=axs[row, column])
    axs[row, column].set_yscale('log')
        
    row += 1
        
axs[0,1].legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
out = output[0]

fig, axs = plt.subplots(2, 2)
plt.subplots_adjust(left=0.1,
                    bottom=0.1,
                    right=0.9,
                    top=0.9,
                    wspace=0.4,
                    hspace=0.4)

row = 0
column = 0
for i, var in enumerate(['pt', 'mass','mreg','ddb']):
    h = out['templates'].project('cutflow',var)    
    for cut in ['min', 'met', 'onemuon', 'leptonicW', 'onebjet', 'onefatjet']:
    
        if i == 2:
            row = 0
            column = 1
    
        mplhep.histplot(h[hist.loc(cut), :], label=cut, ax=axs[row, column])
        axs[row, column].set_yscale('log')
        
    row += 1
        
axs[0,1].legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [3]:
fileset = {
    "TTtoLNu2Q" : [
        "root://eosuser.cern.ch//eos/user/a/adlintul/TTtoLNu2Q_TuneCP5_13p6TeV_powheg-pythia8/Run3Summer22/230131_071621/0000/nanoaod_1.root"
    ],
}

events = NanoEventsFactory.from_root(
    fileset['TTtoLNu2Q'][0],
    schemaclass=ScoutingNanoAODSchema
).events()

In [4]:
top = events.GenPart[
        (abs(events.GenPart.pdgId) == 6)
        & (events.GenPart.hasFlags(['fromHardProcess', 'isLastCopy']))
]

In [7]:
w_top = top.distinctChildren[
    (abs(top.distinctChildren.pdgId) == 24)
    & (top.distinctChildren.hasFlags(['isLastCopy']))
]
w_top = ak.flatten(w_top, axis=2)

In [8]:
w_top_is_had = ak.any(
    (abs(w_top.distinctChildren.pdgId) < 7)
    & (w_top.distinctChildren.hasFlags(['isLastCopy']))
, axis=2)

In [10]:
had_top = w_top[w_top_is_had].distinctParent

In [11]:
fatjet = ak.firsts(events.ScoutingFatJet)

In [50]:
near_top, near_top_dr = fatjet.nearest(had_top, axis=None, threshold=0.8, return_metric=True)
near_top_dr = ak.to_numpy(ak.fill_none(near_top_dr, 99))

In [51]:
fatjet["dr_T"] = near_top_dr

In [13]:
b_near_top = near_top.distinctChildren[abs(near_top.distinctChildren.pdgId) == 5]

In [14]:
b_near_top_matched, b_near_top_dr = fatjet.nearest(b_near_top, axis=None, threshold=0.8, return_metric=True)
b_near_top_dr = ak.to_numpy(ak.fill_none(b_near_top_dr, 99))

In [44]:
fatjet["dr_T_b"] = b_near_top_dr

In [15]:
W_near_top = near_top.distinctChildren[(
    (abs(near_top.distinctChildren.pdgId) == 24)
    & (ak.any(abs(near_top.distinctChildren.distinctChildren.pdgId) < 7))
)]

In [ ]:
W_near_top.pdgId

In [16]:
q_W_near_top = ak.flatten(W_near_top.distinctChildren, axis=2)
q_W_near_top_dr = fatjet.delta_r(q_W_near_top)

In [17]:
index_ascend = ak.argsort(q_W_near_top_dr, axis=1)
index_descend = ak.argsort(q_W_near_top_dr, ascending=False, axis=1)

In [18]:
q_W_near_top_ascend = q_W_near_top[index_ascend]
q_W_near_top_descend = q_W_near_top[index_descend]

In [19]:
q_W_near_top_dr_ascend = q_W_near_top_dr[index_ascend]
q_W_near_top_dr_descend = q_W_near_top_dr[index_descend]

In [40]:
fatjet["dr_T_Wq_max"] = ak.fill_none(ak.firsts(q_W_near_top_dr_descend), 99)
fatjet["dr_T_Wq_min"] = ak.fill_none(ak.firsts(q_W_near_top_dr_ascend), 99)
fatjet["dr_T_Wq_max_pdgId"] = ak.fill_none(ak.firsts(q_W_near_top_descend).pdgId, 99)
fatjet["dr_T_Wq_min_pdgId"] = ak.fill_none(ak.firsts(q_W_near_top_ascend).pdgId, 99)

In [56]:
top_matched = ak.fill_none((fatjet["dr_T_b"] < 0.8) & (fatjet["dr_T_Wq_max"] < 0.8), False)
w_matched = ak.fill_none((fatjet["dr_T_b"] >= 0.8) & (fatjet["dr_T_Wq_max"] < 0.8), False)
non_matched = ak.fill_none((~top_matched) & (~w_matched), True)

In [82]:
cat = np.repeat("ERROR",len(fatjet.pt))

In [95]:
cat = ["top_matched" if t else c for c, t in zip(cat, top_matched.to_numpy())]
cat = ["w_matched" if w else c for c, w in zip(cat, w_matched.to_numpy())]
cat = ["non_matched" if n else c for c, n in zip(cat, non_matched.to_numpy())]

In [100]:
fatjet["cat"] = cat